<a href="https://colab.research.google.com/github/PYH1107/generative_ai/blob/main/%EF%BC%A8%EF%BC%B75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1:
importing and installing necessary packages

## 1-1
1. numpy, pandas, matpotlin, seaborn
2. openai, gradio, ollama

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# 安裝必要的套件
!pip install openai gradio ollama

In [ ]:
# xai-grok-sdk: 由社群開發的輕量級 Python SDK，專為與 xAI 的 Grok API 互動而設計
!pip install requests xai-grok-sdk

## 1-2
installing ollama & get secret keys

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama pull gemma3:1b

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 7cd4618c1faf: 100% ▕▏ 815 MB                         
pulling e0a42594d802: 100% ▕▏  358 B                         
pulling dd084c7d92a3: 100% ▕▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕▏   77 B                         
pulling 120007c81bf8: 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
from google.colab import userdata

# 嘗試從 Colab Secrets 獲取 API 金鑰
try:
    grok_api_key = userdata.get('GROK_API_KEY')
    print("成功獲取 Grok API 金鑰")
except:
    grok_api_key = None
    print("警告: 未找到 Grok API 金鑰")

警告: 未找到 Grok API 金鑰


# Part 2:
setting different documentations


In [ ]:
# 創建 config.py 檔案
%%writefile config.py

# 角色設定
CHARACTER_PROMPT = """你是一位專業的分析師，具有以下特性：
1. 你總是將內容分為不同層次，從最高層次往下分析
2. 你會思考每一個層次的邏輯關係
3. 你透過層層提問，將每一個層次串連起來，盡量用等分思維來解讀
4. 最後，你會透過文字由淺入深地總結整體內容
5. 對於數學內容，你會：
   - 清晰解釋並列出公式、概念和方程式
   - 使用 Markdown 中的 tex 語法撰寫數學公式

當用戶詢問任何問題時，你都會遵循以上分析架構來回應。
"""

# 模型設定
OLLAMA_MODEL = "llama3"  # 依您的 Ollama 可用模型調整

Overwriting config.py


In [ ]:
# model_connectors.py
%%writefile model_connectors.py
import requests
from xai_grok_sdk import XAI
from config import CHARACTER_PROMPT

class ModelConnector:
    def __init__(self, api_key=None):
        self.api_key = api_key
        self.character_prompt = CHARACTER_PROMPT

    def generate_response(self, user_input):
        """各子類別需實現此方法"""
        raise NotImplementedError("請在子類別中實作 generate_response 方法。")

class OllamaConnector(ModelConnector):
    def __init__(self, model="llama3"):
        super().__init__()
        self.model = model
        self.base_url = "http://localhost:11434/api/generate"  # Ollama 預設 URL

    def generate_response(self, user_input):
        try:
            full_prompt = f"System: {self.character_prompt}\n\nUser: {user_input}\n\nAssistant:"
            data = {
                "model": self.model,
                "prompt": full_prompt,
                "stream": False
            }
            response = requests.post(self.base_url, json=data)
            response.raise_for_status()
            response_json = response.json()
            return response_json.get("response", "無回應")
        except Exception as e:
            return f"Ollama API 錯誤: {str(e)}"

class GrokConnector(ModelConnector):
    def __init__(self, api_key, model="grok-2-1212"):
        super().__init__(api_key)
        self.client = XAI(api_key=api_key, model=model)

    def generate_response(self, user_input):
        try:
            response = self.client.invoke(
                messages=[
                    {"role": "system", "content": self.character_prompt},
                    {"role": "user", "content": user_input}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Grok API 錯誤: {str(e)}"


Overwriting model_connectors.py


In [ ]:
# 創建 chat_history.py 檔案
%%writefile chat_history.py

class ChatHistory:
    def __init__(self):
        self.history = []

    def add_message(self, role, content):
        self.history.append((role, content))

    def get_history(self):
        return self.history

    def clear_history(self):
        self.history = []

    def format_for_display(self):
        formatted = []
        for role, content in self.history:
            formatted.append((role, content))
        return formatted

Overwriting chat_history.py


In [ ]:
%%writefile main.py

import gradio as gr
from model_connectors import OllamaConnector, GrokConnector
from chat_history import ChatHistory

class ChatbotApp:
    def __init__(self, grok_api_key=None):
        self.grok_api_key = grok_api_key
        self.chat_history = ChatHistory()

        # 初始化連接器
        self.connectors = {
            "Ollama": OllamaConnector(),
            "Grok": GrokConnector(api_key=grok_api_key) if grok_api_key else None
        }

        self.current_model = "Ollama"

    def set_model(self, model_name):
        if model_name in self.connectors and self.connectors[model_name] is not None:
            self.current_model = model_name
            return f"已切換至 {model_name} 模型"
        else:
            return f"錯誤: {model_name} 模型尚未啟用或金鑰未提供"

    def chat(self, user_input, history):
        if not user_input.strip():
            return history

        connector = self.connectors[self.current_model]
        if connector is None:
            response = f"錯誤: {self.current_model} 連接器未正確初始化"
        else:
            response = connector.generate_response(user_input)

        self.chat_history.add_message("User", user_input)
        self.chat_history.add_message("AI", response)

        return self.chat_history.format_for_display()

    def clear_chat(self):
        self.chat_history.clear_history()
        return []

    def launch_app(self):
        with gr.Blocks(title="分層思考 AI 聊天機器人") as app:
            gr.Markdown("# 分層思考分析型 AI 聊天機器人")

            with gr.Row():
                model_radio = gr.Radio(
                    ["Ollama", "Grok"],
                    label="選擇 AI 模型",
                    value="Ollama"
                )
                model_status = gr.Textbox(label="模型狀態", value="使用 Ollama 模型中")

            chatbot = gr.Chatbot(label="對話", height=500)

            with gr.Row():
                user_input = gr.Textbox(label="輸入您的問題", placeholder="請輸入您想分析的問題...", lines=2)
                clear_btn = gr.Button("清除對話")

            model_radio.change(
                fn=self.set_model,
                inputs=model_radio,
                outputs=model_status
            )

            user_input.submit(
                fn=self.chat,
                inputs=[user_input, chatbot],
                outputs=chatbot
            ).then(
                fn=lambda: "",
                outputs=user_input
            )

            clear_btn.click(
                fn=self.clear_chat,
                outputs=chatbot
            )

        return app

# 啟動點
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="啟動 AI 聊天機器人")
    parser.add_argument("--grok_key", type=str, help="Grok API 金鑰", default=None)

    args = parser.parse_args()

    app = ChatbotApp(
        grok_api_key=args.grok_key
    )
    app.launch_app().launch()


Overwriting main.py


In [ ]:
# 創建 app.py 檔案 - 主要執行檔
%%writefile app.py

from google.colab import userdata
from main import ChatbotApp

try:
    grok_api_key = userdata.get('groq')
except:
    grok_api_key = None
    print("警告: 未找到 Grok API 金鑰")

# 創建並啟動應用
app = ChatbotApp(
    grok_api_key=grok_api_key
)

# 啟動 Gradio 應用
app.launch_app().launch(share=True)

Overwriting app.py


In [ ]:
# 檢查所有文件是否已正確生成
!ls -la

# 執行應用
!python app.py

total 52
drwxr-xr-x 1 root root 4096 May  6 15:46 .
drwxr-xr-x 1 root root 4096 May  6 15:25 ..
-rw-r--r-- 1 root root  323 May  6 16:14 app.py
-rw-r--r-- 1 root root  442 May  6 16:14 chat_history.py
drwxr-xr-x 4 root root 4096 May  2 13:33 .config
-rw-r--r-- 1 root root  672 May  6 16:14 config.py
drwxr-xr-x 2 root root 4096 May  6 15:46 .gradio
-rw-r--r-- 1 root root 3037 May  6 16:14 main.py
-rw-r--r-- 1 root root 1820 May  6 16:14 model_connectors.py
-rw------- 1 root root 4463 May  6 16:14 nohup.out
drwxr-xr-x 2 root root 4096 May  6 15:45 __pycache__
drwxr-xr-x 1 root root 4096 May  2 13:33 sample_data
警告: 未找到 Grok API 金鑰
/content/main.py:57: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="對話", he